In [35]:
import numpy as np
import scipy as sp
from joblib import dump
from pathlib import Path 

In [39]:
n = 10000

arr = np.zeros((n, 1))
arr[0] = 1

In [40]:
arr.nbytes

80000

In [4]:
import polars as pl

def calcbytes(s):
    arr = np.zeros((s, 1))
    arr[s-1] = 1
    dump(arr, 'tmp.pickle')
    return Path('tmp.pickle').stat().st_size

s = [10, 50, 100, 500, 1000, 5000, 10000, 50000, 100000]
b = [calcbytes(_) for _ in s]

pl.DataFrame({'size': s, 'bytes': b}).plot('size', 'bytes')

:Curve   [size]   (bytes)

In [50]:
sp.sparse.coo_matrix(np.zeros((100, 10, 20)))

TypeError: expected dimension <= 2 array or matrix

In [46]:
def calcbytes_sparse(s, kind='csr_array'):
    mapping = {
        'csr_array': sp.sparse.csr_array,
        'coo_array': sp.sparse.coo_array,
    }
    # Create numpy array as normally
    arr = np.zeros((s, 1))
    for i in range(int(s * 0.8 // 1)):
        arr[i] = 1
    
    # Get out a sparse array
    a = mapping[kind](arr)
    dump(a, 'tmp.pickle')
    return Path('tmp.pickle').stat().st_size

df = (
    pl.DataFrame({
        'size': s,
        'np_bytes': [calcbytes(_) for _ in s],
        'csr_bytes': [calcbytes_sparse(_, kind='csr_array') for _ in s],
        'coo_bytes': [calcbytes_sparse(_, kind='coo_array') for _ in s]
    })
)

df

size,np_bytes,csr_bytes,coo_bytes
i64,i64,i64,i64
10,305,607,628
50,625,1151,1140
100,1025,1831,1780
500,4241,7271,6900
1000,8241,14071,13300
5000,40241,68471,64500
10000,80241,136471,128500
50000,400241,680471,640500
100000,800241,1360471,1280500


In [56]:
np_arr = np.array([
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
])

sp_arr = sp.sparse.csr_array(np_arr)
sp_arr

<5x10 sparse array of type '<class 'numpy.int64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [57]:
sp_arr.indptr, sp_arr.indices, sp_arr.data

(array([0, 0, 0, 0, 1, 1], dtype=int32), array([7], dtype=int32), array([1]))

In [6]:
def calcbytes_sparse(s, kind='csr_array'):
    mapping = {
        'csr_array': sp.sparse.csr_array,
        'coo_array': sp.sparse.coo_array,
    }
    # Create numpy array as normally
    arr= np.zeros((s, 1))
    arr[:s//2] = 1
    
    # Get out a sparse array
    a = mapping[kind](arr)
    dump(a, 'tmp.pickle')
    return Path('tmp.pickle').stat().st_size

sb = [calcbytes_sparse(_) for _ in s]

df = (
    pl.DataFrame({
        'size': s,
        'np_bytes': [calcbytes(_) for _ in s],
        'csr_bytes': [calcbytes_sparse(_, kind='csr_array') for _ in s],
        'coo_bytes': [calcbytes_sparse(_, kind='coo_array') for _ in s]
    })
)

df

size,np_bytes,csr_bytes,coo_bytes
i64,i64,i64,i64
10,305,575,572
50,625,975,892
100,1025,1463,1284
500,4241,5463,4484
1000,8241,10471,8500
5000,40241,50471,40500
10000,80241,100471,80500
50000,400241,500471,400500
100000,800241,1000471,800500


In [7]:
! wget  https://calmcode.io/static/data/clinc.csv

--2024-05-03 10:59:52--  https://calmcode.io/static/data/clinc.csv
Resolving calmcode.io (calmcode.io)... 162.159.140.98, 172.66.0.96
Connecting to calmcode.io (calmcode.io)|162.159.140.98|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1265609 (1.2M) [application/octet-stream]
Saving to: ‘clinc.csv.1’

clinc.csv.1         100%[===================>]   1.21M  5.28MB/s    in 0.2s    

2024-05-03 10:59:52 (5.28 MB/s) - ‘clinc.csv.1’ saved [1265609/1265609]



In [8]:
texts = pl.read_csv("clinc.csv")['text'].to_list()

In [9]:
texts[:5]

['how would you say fly in italian',
 "what's the spanish word for pasta",
 'how would they say butter in zambia',
 'how do you say fast in spanish',
 "what's the word for trees in norway"]

In [67]:
def generator():
    yield 'actual'
    yield 'bear'
    yield 'cinematic'

g = generator()

In [68]:
cv = CountVectorizer().fit(g)

In [69]:
cv.vocabulary_

{'actual': 0, 'bear': 1, 'cinematic': 2}

In [62]:
from sklearn.feature_extraction.text import CountVectorizer

out = CountVectorizer().fit_transform(texts).todense()
dump(out, 'tmp.pickle')
Path('tmp.pickle').stat().st_size

1371377041

In [63]:
1371377041/2_245_691

610.6704087962236

In [61]:
179201/23700/7233

0.0010453786297099383

In [60]:
out.todense

<23700x7233 sparse matrix of type '<class 'numpy.int64'>'
	with 179201 stored elements in Compressed Sparse Row format>